In [5]:
account_id_map = {
    'us-east-1': '785573368785',
    'us-east-2': '007439368137',
    'us-west-1': '710691900526',
    'us-west-2': '301217895009',
    'eu-west-1': '802834080501',
    'eu-west-2': '205493899709',
    'eu-west-3': '254080097072',
    'eu-north-1': '601324751636',
    'eu-south-1': '966458181534',
    'eu-central-1': '746233611703',
    'ap-east-1': '110948597952',
    'ap-south-1': '763008648453',
    'ap-northeast-1': '941853720454',
    'ap-northeast-2': '151534178276',
    'ap-southeast-1': '324986816169',
    'ap-southeast-2': '355873309152',
    'cn-northwest-1': '474822919863',
    'cn-north-1': '472730292857',
    'sa-east-1': '756306329178',
    'ca-central-1': '464438896020',
    'me-south-1': '836785723513',
    'af-south-1': '774647643957'
}

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash

In [ ]:
!sudo yum install git-lfs -y

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/microsoft/bloom-deepspeed-inference-int8/
    

In [ ]:
!cd bloom-deepspeed-inference-int8 && git lfs pull

In [ ]:
!cd bloom-deepspeed-inference-int8 && git pull

In [ ]:
!pwd


#### Start triton offering

In [6]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        7.7G     0  7.7G   0% /dev
tmpfs           7.7G  4.0K  7.7G   1% /dev/shm
tmpfs           7.7G  608K  7.7G   1% /run
tmpfs           7.7G     0  7.7G   0% /sys/fs/cgroup
/dev/nvme0n1p1  140G   89G   52G  64% /
/dev/nvme2n1    985G  357G  578G  39% /home/ec2-user/SageMaker
tmpfs           1.6G     0  1.6G   0% /run/user/1000
tmpfs           1.6G     0  1.6G   0% /run/user/1001
tmpfs           1.6G     0  1.6G   0% /run/user/1002


### Start Triton Server

**Note**: Amazon SageMaker expects the model tarball file to have a top level directory with the same name as the model defined in the `config.pbtxt`. Below is the sample model directory structure

```
bert-gptj
├── 1
│   └── model.pt
└── config.pbtxt
```

tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude="*.bin" -exclude "*.tar" -zcvf model.tar.gz bert-gptj



In [7]:
!mkdir -p ./bloom-176b
!mkdir -p ./bloom-176b/1

In [8]:
%%writefile ./bloom-176b/1/model.pt
print("hello model pt")

Overwriting ./bloom-176b/1/model.pt


In [9]:
%%writefile ./bloom-176b/config.pbtxt

name: "t5"
backend: "python"
max_batch_size: 8
input [
  {
    name: "input"
    data_type: TYPE_INT32
    dims: [ -1 ]
    
  }
]
output [
  {
    name: "output"
    data_type: TYPE_INT32
    dims: [ -1 ]
  }
]
instance_group {
  count: 1
  kind: KIND_GPU
}


Overwriting ./bloom-176b/config.pbtxt


In [10]:
%%writefile ./bloom-176b/1/model.py

import numpy as np
import sys
import os
import json
from pathlib import Path

import torch

import triton_python_backend_utils as pb_utils

class TritonPythonModel:
    # Every Python model must have "TritonPythonModel" as the class name!
    def initialize(self, args):
        print(f"TritonPythonModel:Bloom-176b:initialize called:args={args}")
        self.output_dtype = pb_utils.triton_string_to_numpy(
            pb_utils.get_output_config_by_name(
                json.loads(args['model_config']), "output"
            )['data_type']
        )
        print(f"TritonPythonModel:Bloom-176b:initialize::output_type={self.output_dtype}")
        
        #from transformers import T5ForConditionalGeneration, T5Tokenizer
        self.model = None #T5ForConditionalGeneration.from_pretrained("t5-small").cuda()
        print("TritonPythonModel initialized")

    def execute(self, requests):
        print(f"TritonPythonModel:Bloom-176b: execute called:requests={requests}:")
        
        responses = []
        try:
            for request in requests:
                input = pb_utils.get_input_tensor_by_name(request, "input")
                input_ids = input.as_numpy()

                input_ids = torch.as_tensor(input_ids).long().cuda()
                summary = [1,2,3] # self.model.generate(input_ids, num_beams=1)

                # Convert to numpy array on cpu:
                np_summary = summary.cpu().int().detach().numpy()
                inference_response = pb_utils.InferenceResponse(
                    output_tensors=[
                        pb_utils.Tensor(
                            "output",
                            np_summary.astype(self.output_dtype)
                        )
                    ]
                )
                responses.append(inference_response)
        except Exception as e:
            print(f"TritonPythonModel:Bloom-176b: execute:Exception Thrown")
            print (e.message, e.args)
            return responses

    def finalize(self):
        print(f"TritonPythonModel:Bloom-176b: finalize()::TritonPythonModel finalized: called")


Overwriting ./bloom-176b/1/model.py


In [11]:
!cd bloom-176b && rm model.tar.gz
!rm model.tar.gz
!tar --exclude=".git" --exclude=".gitattributes" --exclude="model.tar.gz" --exclude=".bin" --exclude ".tar" -zcvf model.tar.gz bloom-176b  
!tar -tvf model.tar.gz


rm: cannot remove ‘model.tar.gz’: No such file or directory
bloom-176b/
bloom-176b/1/
bloom-176b/1/model.py
bloom-176b/1/model.pt
bloom-176b/config.pbtxt
drwxrwxr-x ec2-user/ec2-user 0 2022-10-10 05:34 bloom-176b/
drwxrwxr-x ec2-user/ec2-user 0 2022-10-10 05:26 bloom-176b/1/
-rw-rw-r-- ec2-user/ec2-user 2112 2022-10-10 16:22 bloom-176b/1/model.py
-rw-rw-r-- ec2-user/ec2-user   24 2022-10-10 16:22 bloom-176b/1/model.pt
-rw-rw-r-- ec2-user/ec2-user  260 2022-10-10 16:22 bloom-176b/config.pbtxt


In [12]:
!aws s3 cp ./model.tar.gz s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b  
    
    

upload: ./model.tar.gz to s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b


#### Start creating the end point

In [14]:
import sagemaker
from sagemaker import get_execution_role, Session, image_uris
from sagemaker.utils import name_from_base
import boto3
region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")
import sagemaker
import boto3

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
region = boto3.Session().region_name

sm_client = boto3.Session().client(service_name="sagemaker", region_name=region)
s3_client = boto3.Session().client(service_name="s3", region_name=region)

In [15]:
s3_model_path_triton = sagemaker.s3.S3Uploader().upload(
    local_path="./model.tar.gz",
    desired_s3_uri="s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b",
    sagemaker_session=session
)
s3_mme_model_path='s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b'
print(s3_model_path_triton)
print(s3_mme_model_path)

s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b/model.tar.gz
s3://sagemaker-us-east-1-622343165275/bloom/triton_models/bloom-176b


In [16]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
from sagemaker.utils import name_from_base

region = boto3.Session().region_name
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
triton_image_uri = "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
    account_id=account_id_map[region], region=region, base=base
)
print(triton_image_uri)

max_seq_length=64

785573368785.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:22.07-py3


In [17]:
from sagemaker.utils import name_from_base

In [18]:
endpoint_name_p5 = name_from_base(f"p5-bert-uc-")
print(endpoint_name_p5)

container_p5 = {
    'Image': triton_image_uri,
    'ModelDataUrl': s3_model_path_triton,
    'Environment': {
        #'SAGEMAKER_PROGRAM' : 'inference.py',
        #'SAGEMAKER_SUBMIT_DIRECTORY' : 'code',
        'SAGEMAKER_TRITON_DEFAULT_MODEL_NAME': 'bloom-176b',
        "SAGEMAKER_TRITON_BATCH_SIZE": "16",
        "SAGEMAKER_TRITON_MAX_BATCH_DELAY": "1000",
        "SAGEMAKER_TRITON_SHM_DEFAULT_BYTE_SIZE" : "16777216", #"16777216000",
        "SAGEMAKER_TRITON_SHM_GROWTH_BYTE_SIZE": "1048576"
    }
}
create_model_response = sm_client.create_model(
    ModelName=endpoint_name_p5, ExecutionRoleArn=role, PrimaryContainer=container_p5
)
print(create_model_response)

p5-bert-uc--2022-10-10-16-22-59-716
{'ModelArn': 'arn:aws:sagemaker:us-east-1:622343165275:model/p5-bert-uc--2022-10-10-16-22-59-716', 'ResponseMetadata': {'RequestId': 'b17387d3-74ce-4187-bb47-d2a1c727f932', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b17387d3-74ce-4187-bb47-d2a1c727f932', 'content-type': 'application/x-amz-json-1.1', 'content-length': '97', 'date': 'Mon, 10 Oct 2022 16:22:59 GMT'}, 'RetryAttempts': 0}}


In [19]:

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_name_p5,
    ProductionVariants = [{
        'InstanceType'        : 'ml.g4dn.xlarge',
        'InitialVariantWeight': 1,
        'InitialInstanceCount': 1,
        'ModelName'           : endpoint_name_p5,
        'VariantName'         : 'AllTraffic'}])

create_endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:622343165275:endpoint-config/p5-bert-uc--2022-10-10-16-22-59-716',
 'ResponseMetadata': {'RequestId': 'b705cdd1-f83b-4a41-89d7-a916d56a6b8f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b705cdd1-f83b-4a41-89d7-a916d56a6b8f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '116',
   'date': 'Mon, 10 Oct 2022 16:22:59 GMT'},
  'RetryAttempts': 0}}

In [20]:
# Create the end point
create_endpoint_response = sm_client.create_endpoint(
    EndpointName         = endpoint_name_p5,
    EndpointConfigName   = endpoint_name_p5)

create_endpoint_response

In [21]:
create_endpoint_response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:622343165275:endpoint/p5-bert-uc--2022-10-10-16-22-59-716',
 'ResponseMetadata': {'RequestId': '5d5e32e0-60f2-4476-8626-29c5028d8b07',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5d5e32e0-60f2-4476-8626-29c5028d8b07',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Mon, 10 Oct 2022 16:23:03 GMT'},
  'RetryAttempts': 0}}

In [ ]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name_p5)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install nvidia-pyindex
!pip install tritonclient[http]
!pip install torch

In [1]:
import numpy as np
from transformers import T5Tokenizer

import tritonclient.http as httpclient

tokenizer = T5Tokenizer.from_pretrained("t5-small")
# print(tokenizer)
input_ids = tokenizer("summarize: SageMaker enables customers to deploy a model using custom code with NVIDIA Triton Inference Server. This functionality is available through the development of Triton Inference Server Containers. These containers include NVIDIA Triton Inference Server, support for common ML frameworks, and useful environment variables that let you optimize performance on SageMaker. For a list of all available Deep Learning Containers images, see Available Deep Learning Containers Images. Deep Learning Containers images are maintained and regularly updated with security patches.", return_tensors='pt').input_ids
print(input_ids.numpy().astype(np.int32).shape)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


(1, 115)


In [ ]:
# set for http client

input_data = input_ids.numpy().astype(np.int32)

input_name = 'input'
output_name = "output"
inputs = []
outputs = []
inputs.append(httpclient.InferInput(input_name, input_data.shape, "INT32"))
inputs[0].set_data_from_numpy(input_data, binary_data=True)
outputs.append(
    httpclient.InferRequestedOutput(output_name, binary_data=True))
request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
    inputs, outputs=outputs)

print(header_length)

request_body

In [ ]:
response = sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType='application/vnd.sagemaker-triton.binary+json;json-header-size={}'.format(header_length),
                                  Body=request_body,
                                 TargetModel='model.tar.gz')

# Parse json header size length from the response
header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
header_length_str = response['ContentType'][len(header_length_prefix):]

# Read response body
result = httpclient.InferenceServerClient.parse_response_body(
    response['Body'].read(), header_length=int(header_length_str))
output_data = result.as_numpy(output_name)

In [ ]:
output_data

In [ ]:
decoded_output = tokenizer.decode(
            output_data[0],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False,
        )
decoded_output